In [1]:
import numpy as np
import ray
import matplotlib.pyplot as plt
import time
from utils import *
from procedures import *
# plt.rcParams["font.sans-serif"] = "DejaVu Sans"
# plt.rcParams["mathtext.default"]  = "it"
# plt.rcParams["mathtext.fontset"]  = "dejavusans"

In [2]:
class EMCVGenerator(object):
    def __init__(self, n_alternatives, gamma, lamda, var, best_index=0):
        self.n_alternatives = n_alternatives
        self.gamma = gamma
        self.means = -(np.arange(n_alternatives)-1)/n_alternatives*lamda
        self.means[best_index] = gamma
        self.best_mean = gamma
        self.variances = np.ones(self.n_alternatives)*var
        self.stds = np.sqrt(self.variances)
        
    def get(self, index, n=1):
        return np.random.normal(self.means[index], self.stds[index], n)
        
    def syscount(self):
        return self.n_alternatives

In [3]:
def func1(n):
    return np.log2(n)

def func2(n):
    return np.sqrt(n)

def func3(n):
    return 0.1*n

In [4]:
class EMCVFuncGenerator(object):
    def __init__(self, n_alternatives, gamma, func, var, best_index=0):
        self.n_alternatives = n_alternatives
        self.gamma = gamma
        self.means = -(np.arange(n_alternatives)-1)/n_alternatives*func(n_alternatives)
        self.means[best_index] = gamma
        self.best_mean = gamma
        self.variances = np.ones(self.n_alternatives)*var
        self.stds = np.sqrt(self.variances)
        
    def get(self, index, n=1):
        return np.random.normal(self.means[index], self.stds[index], n)
        
    def syscount(self):
        return self.n_alternatives

In [5]:
num_cpus=96
ray.shutdown()
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

RayContext(dashboard_url='', python_version='3.8.0', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': 'tcp://127.0.0.1:56455', 'raylet_socket_name': 'tcp://127.0.0.1:60179', 'webui_url': '', 'session_dir': 'C:\\Users\\Admin\\AppData\\Local\\Temp\\ray\\session_2023-12-05_12-55-23_730329_70204', 'metrics_export_port': 59243, 'gcs_address': '127.0.0.1:62917', 'address': '127.0.0.1:62917', 'node_id': 'e3776480cf41589d58fe07957d3169d2345e0c17f30d258da69e4fc8'})

In [6]:
n_replications = 1000
 
min_power = 3
max_power = 16
base = 2

gamma = 0.1
var = 1
lamda = -1

nsd = 2
n0 = 16
ng = 4

In [7]:
EFG_results = []

seed = 2023112203
np.random.seed(seed)
rng = np.random.default_rng(seed)

for func in [func3]:
    for power in np.arange(min_power, max_power+1):
        print("This problem has {} alternatives".format(base**power))
        generators =   [EMCVFuncGenerator(base**power, gamma, func, var) for i in range(n_replications)]
        PCS, PGS = parallel_experiments(rng, generators, policy=EFG, remote_policy=remote_EFG, args={"n0":n0, "ng":ng})
        EFG_results.append(PCS)
EFG_results

This problem has 8 alternatives
--------New experiments with  k=8----------------------
Start to simulate... at Tue Dec  5 12:55:33 2023
PCS:0.521, PGS:0.521
Total time used: 1.8409287929534912s, simulation ends at Tue Dec  5 12:55:33 2023
This problem has 16 alternatives
--------New experiments with  k=16----------------------
Start to simulate... at Tue Dec  5 12:55:35 2023
PCS:0.585, PGS:0.585
Total time used: 1.794384241104126s, simulation ends at Tue Dec  5 12:55:35 2023
This problem has 32 alternatives
--------New experiments with  k=32----------------------
Start to simulate... at Tue Dec  5 12:55:37 2023
PCS:0.553, PGS:0.553
Total time used: 1.8331565856933594s, simulation ends at Tue Dec  5 12:55:37 2023
This problem has 64 alternatives
--------New experiments with  k=64----------------------
Start to simulate... at Tue Dec  5 12:55:38 2023
PCS:0.632, PGS:0.632
Total time used: 1.8176944255828857s, simulation ends at Tue Dec  5 12:55:38 2023
This problem has 128 alternatives
-

[0.521,
 0.585,
 0.553,
 0.632,
 0.661,
 0.678,
 0.661,
 0.657,
 0.668,
 0.663,
 0.658,
 0.676,
 0.671,
 0.678]

In [9]:
EFGPlus_results = []
for func in [func2]:
    for power in np.arange(min_power, max_power+1):
        print("This problem has {} alternatives".format(base**power))
        k = base**power
        G = min(power, n0)
        generators =   [EMCVFuncGenerator(k, gamma, func, var) for i in range(n_replications)]
        PCS, PGS = parallel_experiments(rng, generators, policy=EFGPlus, 
                                                     remote_policy=remote_EFGPlus, args={"nsd":nsd, 
                                                                                     "n0":n0, 
                                                                                     "ng":ng,
                                                                                    "G":G})
        EFGPlus_results.append(PCS)

This problem has 8 alternatives
--------New experiments with  k=8----------------------
Start to simulate... at Tue Dec  5 13:02:59 2023
PCS:0.654, PGS:0.654
Total time used: 2.0925700664520264s, simulation ends at Tue Dec  5 13:02:59 2023
This problem has 16 alternatives
--------New experiments with  k=16----------------------
Start to simulate... at Tue Dec  5 13:03:01 2023
PCS:0.706, PGS:0.706
Total time used: 2.1230947971343994s, simulation ends at Tue Dec  5 13:03:01 2023
This problem has 32 alternatives
--------New experiments with  k=32----------------------
Start to simulate... at Tue Dec  5 13:03:03 2023
PCS:0.736, PGS:0.736
Total time used: 2.1989009380340576s, simulation ends at Tue Dec  5 13:03:03 2023
This problem has 64 alternatives
--------New experiments with  k=64----------------------
Start to simulate... at Tue Dec  5 13:03:05 2023
PCS:0.781, PGS:0.781
Total time used: 2.0890324115753174s, simulation ends at Tue Dec  5 13:03:05 2023
This problem has 128 alternatives


In [ ]:
results = [EFGPlus_results, EFG_results]

In [13]:
results

[[0.654,
  0.706,
  0.736,
  0.781,
  0.796,
  0.865,
  0.903,
  0.916,
  0.931,
  0.954,
  0.964,
  0.979,
  0.984,
  0.995],
 [0.521,
  0.585,
  0.553,
  0.632,
  0.661,
  0.678,
  0.661,
  0.657,
  0.668,
  0.663,
  0.658,
  0.676,
  0.671,
  0.678]]